# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. 

Задача: выяснить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных
Представлены данные по количеству детей, опыту работы, возрасту, уровню образования, семейному статусу, полу, долгу по кредиту, цели кредита и дохода клиентов банка. 

На первый взгляд наблюдается проблема в данных по опыту работы - есть отрицательные значения, чего быть не может, а также не понятно в каких величинах опыт работы измеряется (дни, часы или года). В столбце "education" значения записаны не единообразно, необходимо их привести к строчному варианту. В столбце "purpose" одинаковые цели записаны по разному (неявные дубликаты), необходимо поделить их на одни категории. В столбцах "days_employed" и "total_income" есть пропущенные значения.

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
from pymystem3 import Mystem
m = Mystem()
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков
После вызова функции info обнаружены пропуски значений в столбцах "days_employed" и "total_income" (по 2174 значения пропущено в каждом столбце). Данные пропуски относятся к MNAR (Missing Not At Random) пропускам, где данные отсутствуют в зависимости от неизвестных факторов. 
Восстановить по имеющимся другим данным в датасете невозможно. Значения в данных столбцах относятся к типу float64. Лучшим способом заполнить пропуски будет - заменить на медианное значение по столбцу. 

In [3]:
data.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
days_employed_median = data['days_employed'].median() 
total_income_median = data['total_income'].median() 
data['days_employed'].fillna(days_employed_median, inplace = True)
data['total_income'].fillna(total_income_median, inplace = True)
data.isna().sum() 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.
В столбце days_employed найдены артефакты - в некоторых случаях стаж отрицательный, в остальных он неправдоподобно большой. Скорее всего, база данных была ошибочно заполнена, либо данные были неправильно извлечены. Удалить столбец будет наилучшим выходом, так как для дальнейшего исследования он не понадобится. 
В столбце "dob_years" были обнаружены значения равные 0, что невозможно, так как в данном столбце хранятся данные о возрасте клиентов. Поэтому нулевые значения были заменены на среднее значение по столбцу.
В столбце с числом детей "children" обнаружены неправдопадобные значения - отрицательные значения и слишком высокие значения. Но доля таких значений незначительна (около 1% от всех строк). Для удобства исследования, строки с неверными значениями удалены.

In [5]:
data.drop(['days_employed'], axis='columns', inplace=True)  

In [6]:
data['dob_years'].value_counts().tail(15)

66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [7]:
data.loc[data['dob_years'] == 0] = int(data['dob_years'].mean())
data.loc[data['dob_years'] == 0].count().sum()

0

In [8]:
data['children'].value_counts()

 0     14080
 1      4802
 2      2042
 3       328
 43      101
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [9]:
data = data.loc[data['children'] >= 0]
data = data.loc[data['children'] < 20]
data['children'].value_counts()

0    14080
1     4802
2     2042
3      328
4       41
5        9
Name: children, dtype: int64

### Шаг 2.3. Изменение типов данных.
Данные в столбце "total_income" были типа float64. Для удобства расчетов были переведены в тип int (целочисленных значений). 

In [10]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21302 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21302 non-null  int64 
 1   dob_years         21302 non-null  int64 
 2   education         21302 non-null  object
 3   education_id      21302 non-null  int64 
 4   family_status     21302 non-null  object
 5   family_status_id  21302 non-null  int64 
 6   gender            21302 non-null  object
 7   income_type       21302 non-null  object
 8   debt              21302 non-null  int64 
 9   total_income      21302 non-null  int64 
 10  purpose           21302 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


In [11]:
education_dict = data[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict.head(10)

,education_id,education
0,0,высшее
1,1,среднее
2,1,Среднее
3,1,СРЕДНЕЕ
4,0,ВЫСШЕЕ
5,2,неоконченное высшее
6,3,начальное
7,0,Высшее
8,2,НЕОКОНЧЕННОЕ ВЫСШЕЕ
9,2,Неоконченное высшее


In [12]:
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


### Шаг 2.4. Удаление дубликатов.
В данных были обнаружены явные дубликаты (54) с помощью метода duplicated(), они были удалены с сохранением индексации. Также были обнаружены неявные дубликаты в столбце "education" после применения функции value_counts() для поиска уникальных значений. Элементы в столбце были записаны в разных регистрах. Причиной появляения данных дубликатов возможно неверное заполнение данных изначально. Данные значения были приведены в единый регистр с помощью функции str.lower(). 

In [13]:
data.duplicated().sum()

54

In [14]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

In [15]:
data['education_id'].value_counts()

1    15028
0     5193
2      739
3      282
4        6
Name: education_id, dtype: int64

In [16]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15028
высшее                  5193
неоконченное высшее      739
начальное                282
ученая степень             6
Name: education, dtype: int64

### Шаг 2.6. Категоризация дохода.
Клиенты банка были поделены на 5 групп в зависимости от дохода. Для этого была составлена функция с заданными параметрами для каждой группы дохода. Также был сделан отдельный срез с общим доходом, наличием долга и группами по доходам для дальнейшего анализа.

In [17]:
def find_income_groups(income):
    if income < 30000:
        return('E')
    if 30001 <= income <= 50000:
        return('D')
    if 50001 <= income <= 200000:
        return('C')
    if 200001 <= income <= 1000000:
        return('B')
    if income > 1000001:
        return('A')

In [18]:
clients_income = data[['total_income', 'debt']]
clients_income['income_groups'] = clients_income['total_income'].apply(find_income_groups)
display(clients_income)

/tmp/ipykernel_98/2722038237.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clients_income['income_groups'] = clients_income['total_income'].apply(find_income_groups)


,total_income,debt,income_groups
0,253875,0,B
1,112080,0,C
2,145885,0,C
3,267628,0,B
4,158616,0,C
...,...,...,...
21243,224791,0,B
21244,155999,0,C
21245,89672,1,C
21246,244093,1,B


### Шаг 2.7. Категоризация целей кредита.
В базе данных много целей получения кредита, но можно выделить 4 основных: автомобиль, недвижимость, свадьба, образование. Лемматизируем значения purpose с помощью библиотеки pymystem3, а после добавим новый столбец purpose_category с категориями целей, чтобы упростить анализ.

In [19]:
data['purpose'].value_counts()

свадьба                                   787
на проведение свадьбы                     764
сыграть свадьбу                           759
операции с недвижимостью                  669
покупка коммерческой недвижимости         656
покупка жилья для сдачи                   648
операции с коммерческой недвижимостью     643
операции с жильем                         641
покупка жилья для семьи                   636
жилье                                     635
покупка жилья                             634
недвижимость                              627
строительство собственной недвижимости    626
операции со своей недвижимостью           623
строительство недвижимости                619
строительство жилой недвижимости          618
покупка своего жилья                      618
покупка недвижимости                      613
ремонт жилью                              602
покупка жилой недвижимости                599
на покупку своего автомобиля              501
заняться высшим образованием      

In [20]:
def find_category(row):
    lemmas = m.lemmatize(row)
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'образование'in lemmas:
        return 'образование'
    return 'недвижимость'

data['purpose_category'] = data['purpose'].apply(find_category)
data.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


### Ответы на вопросы.

##### Вопрос 1: Зависимость между уровнем дохода и возвратом кредита в срок.
Клиенты были поделены на 5 групп(равных по количеству клиентов) по уровню дохода. В результате подсчета должников по категориям, было выявлено, что уровень дохода не влияет на возврат кредита в срок. Доля должников одинаково низкая по каждому уровню дохода

In [21]:
clients_income.pivot_table(index = 'income_groups', values = ['debt'], aggfunc = ['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
income_groups,,,
A,25,2,0.080000
B,4987,353,0.070784
C,15867,1346,0.084830
D,347,21,0.060519
E,22,2,0.090909


##### Вопрос 2: Зависимость между количеством детей и возвратом кредита в срок
Доля должников с детьми составляет 9,2%, что несколько выше, чем доля должников без детей (7,5%). Таким образом, наличие детей в некоторой стпени снижает вероятность возврата кредита в срок.

In [22]:
debt_children = data[['children', 'debt']]

clients_children = debt_children.loc[debt_children['children'] > 0]
debtors_children = clients_children.loc[clients_children['debt'] == 1]
debtors_children_mean = len(debtors_children) / len(clients_children)

clients_without_children = debt_children.loc[debt_children['children'] == 0]
debtors_without_children = clients_without_children.loc[clients_without_children['debt'] == 1]
debtors_without_children_mean = len(debtors_without_children) / len(clients_without_children)

print('Доля должников с детьми - {:.1%}'.format(debtors_children_mean))
print('Доля должников без детей - {:.1%}'.format(debtors_without_children_mean))

Доля должников с детьми - 9.2%
Доля должников без детей - 7.5%


##### Вопрос 3: Зависимость между семейным положением клиента и возвратом кредита в срок.
В группе "не женат/не замужем" вероятность не возврата кредита выше всего (9,8%). Наиболее низкая вероятность не возврата кредита в группе "вдовец/вдова" (6,6%). 

In [23]:
debtors_family_status = data[['family_status', 'debt']]
debtors_family_status_grouped = debtors_family_status.groupby('family_status').count()
debtors_family_status_grouped['number'] = debtors_family_status.loc[debtors_family_status['debt'] == 1].groupby('family_status').sum()
debtors_family_status_grouped['mean, %'] = debtors_family_status_grouped['number'] / debtors_family_status_grouped['debt'] * 100
debtors_family_status_grouped.sort_values('mean, %', ascending=False)

,debt,number,"mean, %"
family_status,,,
Не женат / не замужем,2780,272,9.784173
гражданский брак,4125,383,9.284848
женат / замужем,12218,923,7.554428
в разводе,1179,84,7.124682
вдовец / вдова,946,62,6.553911


##### Вопрос 4: Зависимость между целью кредита и возвратом в срок.
Чаще всего должниками в абсолюнтых величинах становятся в результате взятия кредита на проведение свадьбы. После групировки данных по категориям целей кредита наибольшая доля должников была зафиксирована в категориях "автомобиль" (9,3%) и "образование" (9,3%).

In [24]:
bank_clients_purpose = data.pivot_table(index=['purpose_category', 'purpose'], values='debt', aggfunc='sum')
bank_clients_purpose.sort_values('debt', ascending=False).head(10)

debt
purpose_category purpose                                    
свадьба          свадьба                                  63
                 на проведение свадьбы                    61
                 сыграть свадьбу                          57
недвижимость     операции с недвижимостью                 55
                 строительство недвижимости               54
                 покупка жилья для сдачи                  52
                 операции с коммерческой недвижимостью    52
образование      получение дополнительного образования    51
автомобиль       сделка с подержанным автомобилем         51
                 сделка с автомобилем                     50

In [25]:
debt_purpose = data[['purpose_category', 'debt']]
debt_purpose_grouped = debt_purpose.groupby('purpose_category').count()
debt_purpose_grouped['number'] = debt_purpose.loc[debt_purpose['debt'] == 1].groupby('purpose_category').sum()
debt_purpose_grouped['mean, %'] = debt_purpose_grouped['number'] / debt_purpose_grouped['debt'] * 100
debt_purpose_grouped[['mean, %']].sort_values('mean, %', ascending=False)
display(debt_purpose_grouped)

,debt,number,"mean, %"
purpose_category,,,
автомобиль,4260,397,9.319249
недвижимость,10707,777,7.256935
образование,3971,369,9.292370
свадьба,2310,181,7.835498


## Общий вывод:

Задачей исследования было выявить, влияет ли семейное положение и количетсво детей на погашение кредита в срок. Также дополнительно было изучено влияние уровня дохода на возвращение кредита в срок, а также цели кредита на возвращение его в срок.

В ходе анализа были сделаны следующие выводы:

1. Уровень дохода не оказывает влияния на возврат кредита в срок. Доля должников оказалась одинаково низкой во всех категориях клиентов по уровню дохода.
2. Доля должников с детьми составляет 9,2%, что несколько выше, чем доля должников без детей (7,5%). Таким образом, наличие детей в некоторой стпени снижает вероятность возврата кредита в срок. 
3. Есть зависимость между возвратом кредита в срок и семейным положением. Выше доля должников среди незамужних/неженатых людей.
4. Выявлена зависимость между целями кредита и возвратом их в срок. Выше доля должников среди тех, кто взял кредит на покупку автомобиля или образование. 

Примечания по качеству данных: данные по опыту работы были неправдоподобными (были выявлены отрицательные значения, не установлена размерность в которой приведены данные), поэтому их пришлось удалить и не использовать в анализе. Также были пропуски в данных по доходу клиентов, они были заполнены медианным значением, в связи с чем, в исследовании есть некоторые допущения. 